In [ ]:
!pip install pytorch-transformers

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import gc
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
device = torch.device("cuda")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Data

In [ ]:
report_data = pd.read_excel('/content/drive/MyDrive/ESG rating/ESG_Dataset_2018&2019.xlsx')

In [ ]:
report_data

,company_year,text,E_label,S_label,G_label,ESG_label,E_rating,S_rating,G_rating,ESG_rating
0,BNK금융지주_2018,Facility (Store) (As of the first half of 2021...,B,B+,NaN,B+,1,1,NaN,1
1,BNK금융지주_2018,"dustry, BGF Retail installed droplet barriers,...",B,B+,NaN,B+,1,1,NaN,1
2,BNK금융지주_2018,Life In addition to creating a physical and ps...,B,B+,NaN,B+,1,1,NaN,1
3,BNK금융지주_2018,processed food BGF Retail BGF Networks Hello N...,B,B+,NaN,B+,1,1,NaN,1
4,BNK금융지주_2018,Greenhouse Gas Emissions · Reducing Plastic/Di...,B,B+,NaN,B+,1,1,NaN,1
...,...,...,...,...,...,...,...,...,...,...
2213,현대제철_2019,2020 76 Safety Inspection with Everyone's Part...,B+,A+,B+,A,1,0,1.0,0
2214,현대제철_2019,"our stakeholders, Hyundai Steel strives to pro...",B+,A+,B+,A,1,0,1.0,0
2215,현대제철_2019,2019) Classification Environment: CO2 reductio...,B+,A+,B+,A,1,0,1.0,0
2216,현대제철_2019,from the Minister of Health and Welfare as an ...,B+,A+,B+,A,1,0,1.0,0


In [ ]:
report_data_ESG = report_data[['text', 'ESG_rating']]

report_data_E = report_data[['text', 'E_rating']]
report_data_S = report_data[['text', 'S_rating']]
report_data_G = report_data[['text', 'G_rating']]

report_data_ESG = report_data_ESG.dropna()
report_data_E = report_data_E.dropna()
report_data_S = report_data_S.dropna()
report_data_G = report_data_G.dropna()


report_data_ESG["ESG_rating"] = report_data_ESG.ESG_rating.apply(int)
report_data_E["E_rating"] = report_data_E.E_rating.apply(int)
report_data_S["S_rating"] = report_data_S.S_rating.apply(int)
report_data_G["G_rating"] = report_data_G.G_rating.apply(int)


In [ ]:
set(report_data_S["S_rating"])

{0, 1, 2}

In [ ]:
set(report_data_E["E_rating"])

{0, 1, 2, 3}

In [ ]:
print("Number of ESG rating data:",len(report_data_ESG))
print("Number of E rating data:",len(report_data_E))
print("Number of S rating data:",len(report_data_S))
print("Number of G rating data:",len(report_data_G))

Number of ESG rating data: 2218
Number of E rating data: 2218
Number of S rating data: 2218
Number of G rating data: 1780


In [ ]:
from sklearn.model_selection import train_test_split
X_train_ESG, X_test_ESG, y_train_ESG, y_test_ESG = train_test_split(list(report_data_ESG['text']), list(report_data_ESG['ESG_rating']), test_size=0.3)
X_train_E, X_test_E, y_train_E, y_test_E = train_test_split(list(report_data_E['text']), list(report_data_E['E_rating']), test_size=0.3)
X_train_S, X_test_S, y_train_S, y_test_S = train_test_split(list(report_data_S['text']), list(report_data_S['S_rating']), test_size=0.3)
X_train_G, X_test_G, y_train_G, y_test_G = train_test_split(list(report_data_G['text']), list(report_data_G['G_rating']), test_size=0.3)

In [ ]:
model_name = "bert-base-uncased"
max_length = 512

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

# Classification

In [ ]:
from transformers import BertTokenizerFast, BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.cuda()

from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="output",
    num_train_epochs = 3,
    per_device_train_batch_size = 16,  # batch size per device during training
    weight_decay = 0.01,               # strength of weight decay
    load_best_model_at_end = True,
    logging_steps = 50,
    evaluation_strategy = "steps",
    metric_for_best_model= "accuracy"
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

## ESG

### Train

In [ ]:
train_encodings_ESG = tokenizer(X_train_ESG, truncation=True, padding=True, max_length=max_length)
test_encodings_ESG = tokenizer(X_test_ESG, truncation=True, padding=True, max_length=max_length)

In [ ]:
train_dataset_ESG = Dataset(train_encodings_ESG, y_train_ESG)
val_dataset_ESG = Dataset(test_encodings_ESG, y_test_ESG)

In [ ]:
trainer_ESG = Trainer(
    model=model,
    eval_dataset=val_dataset_ESG,
    args=training_args,
    train_dataset=train_dataset_ESG,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [ ]:
trainer_ESG.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1552
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 291


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.754100,0.647241,0.648649,0.418564,0.428152,0.420471
100,0.594200,0.534586,0.755255,0.503289,0.500839,0.506989
150,0.453000,0.455893,0.791291,0.733736,0.704435,0.802921
200,0.360600,0.434538,0.797297,0.533091,0.533948,0.541301
250,0.211600,0.444537,0.813814,0.709416,0.763982,0.680565


***** Running Evaluation *****
  Num examples = 666
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 666
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 666
  Batch size = 8
***** Running Evaluation *****
  Num examples = 666
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning

TrainOutput(global_step=291, training_loss=0.43779914567560674, metrics={'train_runtime': 559.567, 'train_samples_per_second': 8.321, 'train_steps_per_second': 0.52, 'total_flos': 1225056072941568.0, 'train_loss': 0.43779914567560674, 'epoch': 3.0})

In [ ]:
trainer_ESG.save_model("model_ESG")

Saving model checkpoint to model_ESG
Configuration saved in model_ESG/config.json
Model weights saved in model_ESG/pytorch_model.bin
tokenizer config file saved in model_ESG/tokenizer_config.json
Special tokens file saved in model_ESG/special_tokens_map.json


### Predict

In [ ]:
model_path = "/content/drive/MyDrive/model_ESG"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)

loading configuration file /content/drive/MyDrive/model_ESG/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /

In [ ]:
test_trainer = Trainer(model)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
test_data = pd.read_excel("/content/drive/MyDrive/ESG rating/ESG_Dataset_2020.xlsx")

In [ ]:
X_test = list(test_data["text"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

In [ ]:
test_dataset = Dataset(X_test_tokenized)

In [ ]:
# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

***** Running Prediction *****
  Num examples = 1965
  Batch size = 8


In [ ]:
test_data["ESG_pred"] = pd.DataFrame(y_pred)

## E

### Train

In [ ]:
train_encodings_E = tokenizer(X_train_E, truncation=True, padding=True, max_length=max_length)
test_encodings_E = tokenizer(X_test_E, truncation=True, padding=True, max_length=max_length)

In [ ]:
train_dataset_E = Dataset(train_encodings_E, y_train_E)
val_dataset_E = Dataset(test_encodings_E, y_test_E)

In [ ]:
trainer_E = Trainer(
    model=model,
    eval_dataset=val_dataset_E,
    args=training_args,
    train_dataset=train_dataset_E,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [ ]:
trainer_E.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1552
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 291


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.978900,0.926327,0.555556,0.178571,0.138889,0.250000
100,0.748600,0.500163,0.831832,0.492691,0.661431,0.481759
150,0.386600,0.273223,0.890390,0.850511,0.939511,0.797471
200,0.236000,0.222553,0.909910,0.914130,0.926620,0.906687
250,0.192500,0.189058,0.917417,0.937097,0.953605,0.922141


***** Running Evaluation *****
  Num examples = 666
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 666
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 666
  Batch size = 8
***** Running Evaluation *****
  Num examples = 666
  Batch size = 8
***** Running Evaluation *****
  Num examples = 666
  Batch size = 8


Training completed. Do not forg

TrainOutput(global_step=291, training_loss=0.45230597073269874, metrics={'train_runtime': 558.5725, 'train_samples_per_second': 8.336, 'train_steps_per_second': 0.521, 'total_flos': 1225067072126976.0, 'train_loss': 0.45230597073269874, 'epoch': 3.0})

In [ ]:
trainer_E.save_model("model_E")

Saving model checkpoint to model_E
Configuration saved in model_E/config.json
Model weights saved in model_E/pytorch_model.bin
tokenizer config file saved in model_E/tokenizer_config.json
Special tokens file saved in model_E/special_tokens_map.json


### Predict

In [ ]:
model_path = "/content/drive/MyDrive/model_E"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=4)

loading configuration file /content/drive/MyDrive/model_E/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_si

In [ ]:
test_trainer = Trainer(model)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred_E = np.argmax(raw_pred, axis=1)

***** Running Prediction *****
  Num examples = 1965
  Batch size = 8


In [ ]:
test_data["E_pred"] = pd.DataFrame(y_pred_E)

In [ ]:
test_data

,company_year,text,E_label,S_label,G_label,ESG_label,E_rating,S_rating,G_rating,ESG_rating,ESG_pred,E_pred
0,BGF리테일_2020,Facility (Store) (As of the first half of 2021...,B+,B+,A,A,1,1,0.0,0,1,1
1,BGF리테일_2020,"dustry, BGF Retail installed droplet barriers,...",B+,B+,A,A,1,1,0.0,0,1,1
2,BGF리테일_2020,Life In addition to creating a physical and ps...,B+,B+,A,A,1,1,0.0,0,1,1
3,BGF리테일_2020,processed food BGF Retail BGF Networks Hello N...,B+,B+,A,A,1,1,0.0,0,1,1
4,BGF리테일_2020,Greenhouse Gas Emissions · Reducing Plastic/Di...,B+,B+,A,A,1,1,0.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1960,효성_2020,related legal standards. Category Unit Hyosung...,A,A+,B+,A,0,0,1.0,0,1,3
1961,효성_2020,"- Subtotal tCO₂eq 546,521 545,248 563,018 GHGs...",A,A+,B+,A,0,0,1.0,0,0,0
1962,효성_2020,has established a policy to encourage green pr...,A,A+,B+,A,0,0,1.0,0,0,1
1963,효성_2020,"151,110 70 Yongyeon 2 Increasing Water Pump Ef...",A,A+,B+,A,0,0,1.0,0,1,1


## S

### Train

In [ ]:
train_encodings_S = tokenizer(X_train_S, truncation=True, padding=True, max_length=max_length)
test_encodings_S = tokenizer(X_test_S, truncation=True, padding=True, max_length=max_length)

In [ ]:
train_dataset_S = Dataset(train_encodings_S, y_train_S)
val_dataset_S = Dataset(test_encodings_S, y_test_S)

In [ ]:
trainer_S = Trainer(
    model=model,
    eval_dataset=val_dataset_S,
    args=training_args,
    train_dataset=train_dataset_S,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [ ]:
trainer_S.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1552
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 291


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.617500,0.541366,0.788288,0.293871,0.262763,0.333333
100,0.461800,0.366368,0.839339,0.465601,0.518430,0.447540
150,0.338200,0.300883,0.852853,0.534372,0.517384,0.564034
200,0.251000,0.273634,0.857357,0.539409,0.522734,0.565938
250,0.178900,0.359194,0.875375,0.537901,0.548144,0.529972


***** Running Evaluation *****
  Num examples = 666
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 666
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 666
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in label

TrainOutput(global_step=291, training_loss=0.3384683550018625, metrics={'train_runtime': 548.0305, 'train_samples_per_second': 8.496, 'train_steps_per_second': 0.531, 'total_flos': 1225056072941568.0, 'train_loss': 0.3384683550018625, 'epoch': 3.0})

In [ ]:
trainer_S.save_model("model_S")

Saving model checkpoint to model_S
Configuration saved in model_S/config.json
Model weights saved in model_S/pytorch_model.bin
tokenizer config file saved in model_S/tokenizer_config.json
Special tokens file saved in model_S/special_tokens_map.json


### Predict

In [ ]:
model_path = "/content/drive/MyDrive/model_S"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)

loading configuration file /content/drive/MyDrive/model_S/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /co

In [ ]:
test_trainer = Trainer(model)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred_S = np.argmax(raw_pred, axis=1)

***** Running Prediction *****
  Num examples = 1965
  Batch size = 8


In [ ]:
test_data["S_pred"] = pd.DataFrame(y_pred_S)

## G

### Train

In [ ]:
train_encodings_G = tokenizer(X_train_G, truncation=True, padding=True, max_length=max_length)
test_encodings_G = tokenizer(X_test_G, truncation=True, padding=True, max_length=max_length)

In [ ]:
train_dataset_G = Dataset(train_encodings_G, y_train_G)
val_dataset_G = Dataset(test_encodings_G, y_test_G)

In [ ]:
trainer_G = Trainer(
    model=model,
    eval_dataset=val_dataset_G,
    args=training_args,
    train_dataset=train_dataset_G,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [ ]:
trainer_G.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1246
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 234


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.859300,0.783148,0.601124,0.250292,0.200375,0.333333
100,0.706300,0.536585,0.790262,0.522962,0.540137,0.522235
150,0.462000,0.318783,0.878277,0.592183,0.595664,0.594456
200,0.275500,0.296797,0.874532,0.590462,0.596122,0.591690


***** Running Evaluation *****
  Num examples = 534
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 534
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 534
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in label

TrainOutput(global_step=234, training_loss=0.5268003634917431, metrics={'train_runtime': 413.836, 'train_samples_per_second': 9.033, 'train_steps_per_second': 0.565, 'total_flos': 983517955467264.0, 'train_loss': 0.5268003634917431, 'epoch': 3.0})

In [ ]:
trainer_G.save_model("model_G")

Saving model checkpoint to model_G
Configuration saved in model_G/config.json
Model weights saved in model_G/pytorch_model.bin
tokenizer config file saved in model_G/tokenizer_config.json
Special tokens file saved in model_G/special_tokens_map.json


### Predict

In [ ]:
model_path = "/content/drive/MyDrive/model_G"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)

loading configuration file /content/drive/MyDrive/model_G/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /co

In [ ]:
test_trainer = Trainer(model)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred_G = np.argmax(raw_pred, axis=1)

***** Running Prediction *****
  Num examples = 1965
  Batch size = 8


In [ ]:
test_data["G_pred"] = pd.DataFrame(y_pred_G)

In [ ]:
test_data

,company_year,text,E_label,S_label,G_label,ESG_label,E_rating,S_rating,G_rating,ESG_rating,ESG_pred,E_pred,G_pred
0,BGF리테일_2020,Facility (Store) (As of the first half of 2021...,B+,B+,A,A,1,1,0.0,0,1,1,1
1,BGF리테일_2020,"dustry, BGF Retail installed droplet barriers,...",B+,B+,A,A,1,1,0.0,0,1,1,1
2,BGF리테일_2020,Life In addition to creating a physical and ps...,B+,B+,A,A,1,1,0.0,0,1,1,1
3,BGF리테일_2020,processed food BGF Retail BGF Networks Hello N...,B+,B+,A,A,1,1,0.0,0,1,1,1
4,BGF리테일_2020,Greenhouse Gas Emissions · Reducing Plastic/Di...,B+,B+,A,A,1,1,0.0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1960,효성_2020,related legal standards. Category Unit Hyosung...,A,A+,B+,A,0,0,1.0,0,1,3,1
1961,효성_2020,"- Subtotal tCO₂eq 546,521 545,248 563,018 GHGs...",A,A+,B+,A,0,0,1.0,0,0,0,0
1962,효성_2020,has established a policy to encourage green pr...,A,A+,B+,A,0,0,1.0,0,0,1,0
1963,효성_2020,"151,110 70 Yongyeon 2 Increasing Water Pump Ef...",A,A+,B+,A,0,0,1.0,0,1,1,1


In [ ]:
test_data.to_csv("test_data.csv")

In [ ]:
(test_data["ESG_rating"] == test_data["ESG_pred"]).value_counts()

False    1088
True      877
dtype: int64

In [ ]:
print("ESG정확도:",(test_data["ESG_rating"] == test_data["ESG_pred"]).value_counts()[1]/((test_data["ESG_rating"] == test_data["ESG_pred"]).value_counts()[0]+(test_data["ESG_rating"] == test_data["ESG_pred"]).value_counts()[1]))

ESG정확도: 0.44631043256997455


In [ ]:
(test_data["E_rating"] == test_data["E_pred"]).value_counts()

True     1288
False     677
dtype: int64

In [ ]:
print("E정확도:",(test_data["E_rating"] == test_data["E_pred"]).value_counts()[1]/((test_data["E_rating"] == test_data["E_pred"]).value_counts()[0]+(test_data["E_rating"] == test_data["E_pred"]).value_counts()[1]))

E정확도: 0.655470737913486


In [ ]:
(test_data["S_rating"] == test_data["S_pred"]).value_counts()

True     1627
False     338
dtype: int64

In [ ]:
print("S정확도:",(test_data["S_rating"] == test_data["S_pred"]).value_counts()[1]/((test_data["S_rating"] == test_data["S_pred"]).value_counts()[0]+(test_data["S_rating"] == test_data["S_pred"]).value_counts()[1]))

S정확도: 0.8279898218829517


In [ ]:
(test_data["G_rating"] == test_data["G_pred"]).value_counts()

False    1189
True      776
dtype: int64

In [ ]:
print("G정확도:",(test_data["G_rating"] == test_data["G_pred"]).value_counts()[1]/((test_data["G_rating"] == test_data["G_pred"]).value_counts()[0]+(test_data["G_rating"] == test_data["G_pred"]).value_counts()[1]))

G정확도: 0.39491094147582695
